In [1]:
import pandas as pd
import sys, os

rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from utils import DatabaseEngine

In [2]:
db_engine = DatabaseEngine()
engine = db_engine.create()
clean_df = pd.read_sql_table("clean_xdr_data", con=engine)

Successfully connected to the PostgreSQL "telecom" database


In [7]:
data_df = pd.read_csv('../data/user_info.csv')

In [10]:
# concatinate the two dataframes
df = pd.concat([clean_df, data_df], axis=1)

#### Relevant variables and associated data types

In [11]:
# relevant variables and associated data types of df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 74 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 150000 non-null  object 
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      150000 non-null  object 
 7   MSISDN/Number                             150000 non-null  object 
 8   IMEI                                      150000 non-null  object 
 9   Last Location Name                        150000 non-null  object 
 10  Avg RTT DL (ms)     

#### Analyzing the basic metrics of the dataset

In [12]:
df.describe()

,Start ms,End ms,Dur. (ms),Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),DL TP < 50 Kbps (%),...,Email Download Data,Email Upload Data,Youtube Download Data,Youtube Upload Data,Netflix Download Data,Netflix Upload Data,Gaming Download Data,Gaming Upload Data,Other Download Data,Other Upload Data
count,150000.000000,150000.000000,1.500000e+05,150000.000000,150000.000000,150000.000000,150000.000000,1.500000e+05,1.500000e+05,150000.000000,...,1.068560e+05,1.068560e+05,1.068560e+05,1.068560e+05,1.068560e+05,1.068560e+05,1.068560e+05,1.068560e+05,1.068560e+05,1.068560e+05
mean,499.188200,498.800880,1.046086e+05,109.795706,17.662883,13300.045927,1770.428647,2.080991e+07,7.596587e+05,92.844754,...,2.497352e+06,6.514435e+05,1.621461e+07,1.534379e+07,1.620639e+07,1.533194e+07,5.882158e+08,1.155326e+07,5.869916e+08,1.151716e+07
std,288.611834,288.097653,8.103762e+04,559.344484,76.530188,23971.878541,4625.355500,1.172360e+08,1.577621e+07,13.005265,...,1.897063e+06,4.948601e+05,1.232294e+07,1.159870e+07,1.231717e+07,1.161060e+07,4.440290e+08,8.731469e+06,4.437790e+08,8.706879e+06
min,0.000000,0.000000,7.142000e+03,0.000000,0.000000,0.000000,0.000000,2.000000e+00,1.000000e+00,0.000000,...,9.700000e+01,1.400000e+01,5.300000e+01,1.060000e+02,4.200000e+01,3.500000e+01,2.516000e+03,5.900000e+01,3.290000e+03,1.560000e+02
25%,250.000000,251.000000,5.744050e+04,35.000000,3.000000,43.000000,47.000000,1.332913e+06,6.300775e+04,91.000000,...,1.184544e+06,3.065678e+05,7.647143e+06,7.246758e+06,7.593534e+06,7.206775e+06,2.793230e+08,5.445849e+06,2.767785e+08,5.461189e+06
50%,499.000000,500.000000,8.639900e+04,54.000000,7.000000,63.000000,63.000000,2.080991e+07,7.596587e+05,100.000000,...,2.266259e+06,5.872975e+05,1.460547e+07,1.386731e+07,1.463991e+07,1.387814e+07,5.325190e+08,1.044654e+07,5.299489e+08,1.037673e+07
75%,749.000000,750.000000,1.324302e+05,109.795706,17.662883,19710.750000,1120.000000,2.080991e+07,7.596587e+05,100.000000,...,3.250651e+06,8.525612e+05,2.112900e+07,1.997808e+07,2.113463e+07,2.000853e+07,7.669561e+08,1.508080e+07,7.653643e+08,1.502752e+07
max,999.000000,999.000000,1.859336e+06,96923.000000,7120.000000,378160.000000,58613.000000,4.294426e+09,2.908226e+09,100.000000,...,3.360721e+07,9.013997e+06,2.378830e+08,2.150757e+08,1.994792e+08,2.032419e+08,7.622035e+09,1.547897e+08,8.025477e+09,1.529305e+08
